<a href="https://colab.research.google.com/github/distractedm1nd/mar2moon/blob/main/scripts/Wav2Vec2_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets==1.4.1
!pip install transformers==4.4.0
!pip install soundfile
!pip install jiwer

     |████████████████████████████████| 194kB 8.0MB/s 
     |████████████████████████████████| 245kB 60.1MB/s 
     |████████████████████████████████| 122kB 67.8MB/s 
     |████████████████████████████████| 2.1MB 8.9MB/s 
     |████████████████████████████████| 3.3MB 60.1MB/s 
     |████████████████████████████████| 901kB 62.9MB/s 
     |████████████████████████████████| 51kB 4.7MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149823 sha256=bf24b6560d15a054ae71fc29a5eaf0bfc95be44806e96b9c8ec7ddaf2ee4dc37
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-Levenshtein


In [1]:
!nvidia-smi

Mon Jun 28 15:11:00 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from datasets import Dataset
from IPython.display import display, HTML
import pandas as pd
import numpy as np
import re
import os
import random

#df = pd.read_excel("../data/Sentiment_Labelling_Altcoin_Daily_210401_to_210531_Batch_1.xlsx")
df = pd.read_csv("data3.csv", sep=";")
df = df.dropna(subset=["Podcast_Title"])[["Podcast_Title", "corrected"]]
df = df[~df.corrected.str.contains(r'[0-9]')]
df["file"] = df.apply(lambda x: f"drive/MyDrive/clips/3/{x['Podcast_Title'][:-7]}_{x.name}.wav" if f"{x['Podcast_Title'][:-7]}_{x.name}.wav" in os.listdir("drive/MyDrive/clips/3") else np.nan, axis=1)
batch = Dataset.from_pandas(df.dropna(subset=["file"])).shuffle().train_test_split(test_size=0.2)
df.dropna(subset=["file"])

,Podcast_Title,corrected,file
0,Altcoin Daily_20210401_ETH to CRUSH Bitcoin (T...,could outperform bitcoin and is watching it ve...,drive/MyDrive/clips/3/Altcoin Daily_20210401_E...
1,Altcoin Daily_20210401_ETH to CRUSH Bitcoin (T...,has a slightly more nuanced opinion than many ...,drive/MyDrive/clips/3/Altcoin Daily_20210401_E...
2,Altcoin Daily_20210401_ETH to CRUSH Bitcoin (T...,to be more um sound from a monetary perspectiv...,drive/MyDrive/clips/3/Altcoin Daily_20210401_E...
3,Altcoin Daily_20210401_ETH to CRUSH Bitcoin (T...,so in bitcoin hard forks are extremely rare in...,drive/MyDrive/clips/3/Altcoin Daily_20210401_E...
4,Altcoin Daily_20210401_ETH to CRUSH Bitcoin (T...,the market but nevertheless the minority a hun...,drive/MyDrive/clips/3/Altcoin Daily_20210401_E...
...,...,...,...
873,Lex_Fridman_20210603_Vitalik_Buterin_-_Ethereu...,like basically squandering that opportunity so...,drive/MyDrive/clips/3/Lex_Fridman_20210603_Vit...
874,Lex_Fridman_20210603_Vitalik_Buterin_-_Ethereu...,and like one example of this in ethereum lands...,drive/MyDrive/clips/3/Lex_Fridman_20210603_Vit...
875,Lex_Fridman_20210603_Vitalik_Buterin_-_Ethereu...,is just as like a psyop by you know the bitcoi...,drive/MyDrive/clips/3/Lex_Fridman_20210603_Vit...
876,Lex_Fridman_20210603_Vitalik_Buterin_-_Ethereu...,i knew that that wasn't entirely true like the...,drive/MyDrive/clips/3/Lex_Fridman_20210603_Vit...


In [4]:
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\d\[\]]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["corrected"]).upper()
    return batch

batch = batch.map(remove_special_characters)

def extract_all_chars(batch):
    all_text = " ".join(batch["text"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

vocabs = batch.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=batch.column_names["train"])

vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{' ': 0,
 "'": 24,
 'A': 21,
 'B': 8,
 'C': 2,
 'D': 18,
 'E': 7,
 'F': 17,
 'G': 12,
 'H': 15,
 'I': 4,
 'J': 14,
 'K': 10,
 'L': 20,
 'M': 11,
 'N': 25,
 'O': 3,
 'P': 1,
 'Q': 9,
 'R': 16,
 'S': 13,
 'T': 23,
 'U': 26,
 'V': 27,
 'W': 19,
 'X': 22,
 'Y': 5,
 'Z': 6}

In [5]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(len(vocab_dict))

30


In [6]:
import json
with open('vocab2.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [7]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab2.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")


In [8]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [9]:
from transformers import Wav2Vec2Processor

#processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)
#processor.save_pretrained("wav2vec2-base-processor")
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")

In [10]:
import soundfile as sf

total_audiolength = 0
def speech_file_to_array_fn(batch):
    global total_audiolength
    speech_array, sampling_rate = sf.read(batch["file"])
    total_audiolength += len(speech_array)
    batch["speech"] = speech_array
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["text"]
    return batch

batch = batch.map(speech_file_to_array_fn, remove_columns=batch.column_names["train"], num_proc=4)
print(total_audiolength)





314880


In [11]:
import IPython.display as ipd
import numpy as np

print(batch["train"][5]["target_text"])
ipd.Audio(data=np.asarray(batch["train"][5]["speech"]), autoplay=False, rate=16000)

ITS SUSPENSION FOR BITCOIN PAYMENTS YESTERDAY SO ADA FOR TESLA CARS YOUR CARDANO FOR TESLA WOULD YOU DO THIS DO YOU THINK THIS IS A GOOD FIT NOW THERE IS


In [12]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values

    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

batch_prepared = batch.map(prepare_dataset, remove_columns=batch.column_names["train"], num_proc=4, batched=True, batch_size=8)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [26]:
import torch
import numpy as np
from datasets import load_metric
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)
wer_metric = load_metric("wer")

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [27]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-960h",
    force_download=True,
    gradient_checkpointing=True,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
)
model.train()

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureExtractor(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (2): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (3): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (4): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (5): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
        )
        (6): Wav2Vec

In [28]:
model.freeze_feature_extractor()

In [29]:
def map_to_result(batch):
    model.to("cuda")
    input_values = processor(
        batch["speech"],
        sampling_rate=batch["sampling_rate"],
        return_tensors="pt"
    ).input_values.to("cuda")

    with torch.no_grad():
        logits = model(input_values).logits

    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_str"] = processor.batch_decode(pred_ids)[0]

    return batch

results = batch["test"].map(map_to_result)
print("WER Before Training: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["target_text"])))

/usr/local/lib/python3.7/dist-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")



WER Before Training: 0.693


In [30]:
from transformers import Trainer
from transformers import TrainingArguments
training_args = TrainingArguments(
  output_dir="./drive/MyDrive/wav2vec2-test3",
  group_by_length=True,
  per_device_train_batch_size=4,
  evaluation_strategy="steps",
  num_train_epochs=10,
  fp16=False,
  learning_rate=1e-5,
  logging_first_step=True,
  weight_decay=0.005,
  save_steps=100,
  eval_steps=100,
  logging_steps=50,
  warmup_steps=350,
  save_total_limit=2,
)


trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=batch_prepared["train"],
    eval_dataset=batch_prepared["test"],
    tokenizer=processor.feature_extractor,
)

In [31]:
trainer.train()

Step,Training Loss,Validation Loss,Wer,Runtime,Samples Per Second
100,0.879200,0.530814,0.270292,39.234700,4.307000
200,0.761300,0.447869,0.249797,39.295100,4.301000
300,0.611500,0.413155,0.240057,39.443800,4.285000
400,0.524700,0.370486,0.214692,39.458100,4.283000
500,0.531400,0.356890,0.212256,39.695900,4.257000
600,0.481100,0.339907,0.201096,39.869300,4.239000
700,0.447900,0.330563,0.198864,39.837100,4.242000
800,0.457500,0.340886,0.193182,39.897400,4.236000
900,0.399100,0.309195,0.189326,40.141100,4.210000
1000,0.408300,0.315237,0.185065,40.119000,4.212000


TrainOutput(global_step=1690, training_loss=0.49141606790779613, metrics={'train_runtime': 3694.5595, 'train_samples_per_second': 0.457, 'total_flos': 2.054141761617888e+18, 'epoch': 10.0, 'init_mem_cpu_alloc_delta': 27809579, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 2685260471, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 1071825, 'train_mem_gpu_alloc_delta': 3760803840, 'train_mem_cpu_peaked_delta': 79505912, 'train_mem_gpu_peaked_delta': 3406102528})

In [32]:
model.eval()
results = batch["test"].map(map_to_result)
print("WER After Training: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["target_text"])))


WER After Training: 0.170


In [33]:
pd.DataFrame(results)[["pred_str", "target_text"]]

,pred_str,target_text
0,S SPOKEN ABOUT MAKING A DECENTRALIZED VIRGION ...,SPOKEN ABOUT MAKING A DECENTRALIZED VERSION OF...
1,THIS TO THE DOGEARMY AND SHOULD DOGE HOLDERS B...,S THIS TO THE DOGE ARMY AND SHOULD DOGE HOLDER...
2,COULD TALK ABOUT FIRST OFALL YOUR POCKASKED AN...,COULD TALK ABOUT FIRST OF ALL YOUR PODCAST AND...
3,N A STATEMENT THAT THE APPLICATION FOR A LICEN...,STATEMENT THAT THE APPLICATION FOR A LICENSE H...
4,THAT KIND OF TECHNOLOGY II DEFINITELY THINK TH...,THAT KIND OF TECHNOLOGY I I DEFINITELY THINK T...
...,...,...
164,UH THIS CENTURY AND SO BITCOIN IS THE FIRST US...,UH THIS CENTURY AND SO BITCOIN IS THE FIRST US...
165,BE THE WAY THE ENTIRE LENDING INDUSTRY WORKS T...,BE THE WAY THE ENTIRE LENDING INDUSTRY WORKS T...
166,AS SIGNIFICANT USE THE COUNTRY TRYING TO TO AC...,IT'S A SIGNIFICANT USER COUNTRY TRYING TO TO A...
167,RE NOT BUYING HERE AND ULTIMATELY AGAIN THIS I...,RE NOT BUYING HERE AND ULTIMATELY AGAIN THIS I...
